# Record Linkage

In [8]:
import recordlinkage
import pandas as pd
import collections
import time
import os
import numpy as np
import random

In [2]:
MEDIATED_DATASETS_PATH = "./Mediated Datasets/"

In [27]:
def get_all_files(path_dir):
    res = []
    for path in os.listdir(path_dir):
        if os.path.isfile(os.path.join(path_dir, path)) and path != '.DS_Store':
            res.append(path)
    return res


def get_features(df_l, df_r):
    # set_index
    indexer = recordlinkage.Index()
    indexer.sortedneighbourhood('name')
    candidate_links = indexer.index(df_l, df_r)
    print('candidate_links:', len(candidate_links))

    # Comparison step
    compare_cl = recordlinkage.Compare()
    compare_cl.string("name", "name", method="jarowinkler", threshold=0.91, label="name")
    compare_cl.exact("ceo", "ceo", missing_value=0.5, label="ceo")

    return compare_cl.compute(candidate_links, df_l, df_r), candidate_links


def get_pairs(features):
    # Classification step
    pairs = features[features.sum(axis=1) > 1]
    maybe = features[features.sum(axis=1) > 0]
    print('pairs:', len(pairs))
    return pairs, maybe


def rename_columns(columns_l, columns_r):
    columns_l = [c + '_l' for c in columns_l]
    columns_r = [c + '_r' for c in columns_r]
    return columns_l, columns_r


def remove_suffix(columns):
    return [c[:-2] for c in columns]


def find_duplicated_columns(columns_l, columns_r):
    attr_cleaned = columns_l + columns_r
    duplicates = [item for item, count in collections.Counter(attr_cleaned).items() if count > 1]
    return duplicates


def join(matching_df_l, matching_df_r):
    matching_df_l = matching_df_l.reset_index(drop=True)
    matching_df_r = matching_df_r.reset_index(drop=True)
    duplicates = find_duplicated_columns(matching_df_l.columns.values.tolist(),
                                         matching_df_r.columns.values.tolist())

    col_l, col_r = rename_columns(matching_df_l.columns.values.tolist(),
                                  matching_df_r.columns.values.tolist())
    matching_df_l.columns = col_l
    matching_df_r.columns = col_r
    joined_df = pd.concat([matching_df_l, matching_df_r], axis=1)
    candidate_joined_df = joined_df.copy()

    for col in duplicates:
        candidates = joined_df[[col + '_l', col + '_r']]
        candidate_labels = candidates.columns.values.tolist()
        #display(candidates.sample(10))
        # time.sleep(1)
        #idx_drop = int(input("Inserisci l'indice (0 o 1) della colonna che vuoi scartare nel dataset finale: "))
        joined_df.drop(candidate_labels[0], axis=1, inplace=True)

    joined_df.columns = [col[:-2] for col in joined_df.columns.values.tolist()]
    return joined_df, candidate_joined_df


def write_annotation(candidate_links, df_l, df_r):
    random_candidates_links = candidate_links.tolist()
    random.shuffle(random_candidates_links)
    x = str(random_candidates_links)
    random_candidates_links = pd.MultiIndex.from_tuples(random_candidates_links)
    set_training_len = int(len(random_candidates_links) * 0.3)
    if write_annotation:
        recordlinkage.write_annotation_file(
            "annotation_linking.json",
            random_candidates_links[0:set_training_len],
            df_l,
            df_r,
            dataset_a_name="df_l",
            dataset_b_name="df_r"
        )


def get_linking_datasets(df_l, df_r, pairs):
    left = []
    right = []

    for elem in pairs:
        left.append(df_l.loc[elem[0]])
        right.append(df_r.loc[elem[1]])

    matching_dataset_left = pd.DataFrame(left)
    matching_dataset_right = pd.DataFrame(right)
    return matching_dataset_left, matching_dataset_right


def get_difference_datasets(df_a, df_b):
    return pd.concat([df_a, df_b]).drop_duplicates(keep=False)


def compute_record_linkage(df_l, df_r):
    print('Compute features and pairs...')
    features, candidate_links  = get_features(df_l, df_r)
    pairs, maybe = get_pairs(features)
    pairs_index = pairs.index
    print('Features and pairs computed successfully')

    ## genero file di annotazione per classificare manualmente le coppie di record calcolate da utilizzare su RecordLinkage ANNOTATOR
    print('Write annotation file')
    #write_annotation(pairs_index, df_l, df_r)
    write_annotation(candidate_links, df_l, df_r)

    print('Compute matching datasets')
    matching_df_l, matching_df_r = get_linking_datasets(df_l, df_r, pairs_index)

    print('Compute difference datasets')
    difference_l = get_difference_datasets(df_l, matching_df_l)
    difference_r = get_difference_datasets(df_r, matching_df_r)

    print('Compute join')
    joined_df, candidate_joined_df = join(matching_df_l, matching_df_r)

    result = pd.concat([joined_df, difference_l])
    result = pd.concat([result, difference_r])
    result = result.reset_index(drop=True)

    multi_index = pd.MultiIndex.from_tuples(random_dio)

    return result, pairs, multi_index


start_time = time.time()

files = get_all_files(MEDIATED_DATASETS_PATH)
df_l = pd.read_json(MEDIATED_DATASETS_PATH + files[0], encoding='utf-8', lines=True, dtype=object)
df_l = df_l.replace(r'^\s*$', np.nan, regex=True)
files.pop(0)
pairs = None
multi_index = None

for file in files:
    df_r = pd.read_json(MEDIATED_DATASETS_PATH + file, encoding='utf-8', lines=True, dtype=object)
    df_r = df_r.replace(r'^\s*$', np.nan, regex=True)
    df_l, pairs, multi_index = compute_record_linkage(df_l, df_r)
    break

df_l

end_time = time.time()
print('Execution time: %s seconds' % (end_time - start_time))

Compute features and pairs...
candidate_links: 1451
pairs: 314
Features and pairs computed successfully
Write annotation file
Compute matching datasets
Compute difference datasets
Compute join
Execution time: 1.145752191543579 seconds


In [28]:
multi_index

MultiIndex([(544,   17),
            (294, 1842),
            (504, 4376),
            ( 24, 6250),
            (173,  925),
            (107, 4021),
            (235, 4299),
            (559, 1234),
            (116, 3818),
            (191, 1351),
            ...
            ( 68, 4050),
            (445, 2669),
            (139, 1058),
            (273, 2990),
            (  8, 6020),
            (216, 1963),
            (468, 6234),
            (320, 2265),
            (105,  661),
            (511, 3037)],
           length=1451)

In [10]:
df_l.drop_duplicates()

,name,headquarters,country,continent,region,sub_region,founded,employees,ceo,market_cap,...,industry,revenue,link,share_price,results_for_year,total_equity,founders,market_value,address,type
0,industrial bank of korea ibk,NaN,south-korea,NaN,NaN,NaN,NaN,NaN,NaN,6760000000.0,...,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN
2,nestle india,NaN,india,NaN,NaN,NaN,NaN,NaN,NaN,23520000000.0,...,NaN,NaN,NaN,243.0,NaN,NaN,NaN,NaN,NaN,NaN
3,atoss,NaN,germany,NaN,NaN,NaN,NaN,NaN,NaN,1430000000.0,...,NaN,NaN,NaN,179.0,NaN,NaN,NaN,NaN,NaN,NaN
5,elisa,NaN,finland,NaN,NaN,NaN,NaN,NaN,NaN,8970000000.0,...,NaN,NaN,NaN,55.0,NaN,NaN,NaN,NaN,NaN,NaN
6,sumitomo chemical india,NaN,india,NaN,NaN,NaN,NaN,NaN,NaN,2950000000.0,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39742,castellum,NaN,usa,NaN,NaN,NaN,NaN,NaN,NaN,51080000.0,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
39743,sporting clube de portugal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58930000.0,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
39744,huhtamaki india,NaN,india,NaN,NaN,NaN,NaN,NaN,NaN,180000000.0,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
39745,digital bros,NaN,italy,NaN,NaN,NaN,NaN,NaN,NaN,350000000.0,...,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
pairs

,,name,ceo
154,1377,1.0,0.5
9687,1377,1.0,0.5
335,2370,1.0,0.5
337,4110,1.0,0.5
9491,4110,1.0,0.5
...,...,...,...
39832,5829,1.0,0.5
39837,4083,1.0,0.5
39840,6674,1.0,0.5
39841,6674,1.0,0.5


In [4]:
start_time = time.time()

files = get_all_files(MEDIATED_DATASETS_PATH)
df_l = pd.read_json(MEDIATED_DATASETS_PATH + files[0], encoding='utf-8', lines=True, dtype=object)
df_l = df_l.replace(r'^\s*$', np.nan, regex=True)
files.pop(0)
pairs = None

for file in files:
    df_r = pd.read_json(MEDIATED_DATASETS_PATH + file, encoding='utf-8', lines=True, dtype=object)
    df_r = df_r.replace(r'^\s*$', np.nan, regex=True)
    df_l, pairs = compute_record_linkage(df_l, df_r)
    df_l.reset_index(drop=True)

df_l

end_time = time.time()
print('Execution time: %s seconds' % (end_time - start_time))

Compute features and pairs...
candidate_links: 21282
pairs: 7334
Features and pairs computed successfully
Write annotation file
Compute matching datasets
Compute difference datasets
Compute join
Compute features and pairs...


ValueError: index of DataFrame is not unique

## Classification

In [52]:
##all'interno della funzione read_annotation_file metti il nome del file che esport da recordlinkage annotator
result_json = recordlinkage.read_annotation_file('result_30%.json')
true_links = result_json.links  ##record etichettati con match
false_links = result_json.distinct  ##record etichettati con distinct
print('links:', len(true_links))
print('distinct:', len(false_links))

links: 112
distinct: 323


In [53]:
lung = int(len(multi_index)*0.3)
golden_pairs = pairs[0:lung]
golden_match_index = golden_pairs.index.intersection(true_links)
len(golden_pairs)

314

In [60]:
pairs

,,name,ceo
28,1136,1.0,0.5
64,385,1.0,0.5
71,6458,1.0,0.5
113,2695,1.0,0.5
130,708,1.0,0.5
139,2322,1.0,0.5
279,4899,1.0,0.5
313,1369,1.0,0.5
315,2993,1.0,0.5
353,2306,1.0,0.5


multi_index

In [54]:
golden_match_index

MultiIndex([(313, 1369),
            (573, 1119),
            (575, 1112),
            (583, 3285),
            (  5,  457),
            ( 18, 2803),
            ( 24,  603),
            ( 32,  148),
            ( 34, 1126),
            ( 40,  897),
            ( 45, 3348),
            ( 50, 1908),
            ( 52,   85),
            ( 56,  201),
            ( 58,  479),
            ( 68,  205),
            ( 78,  293),
            ( 90,  230),
            ( 95,   98),
            (104, 1451),
            (105, 1052),
            (111,  578),
            (113,  452),
            (122, 2485),
            (133,   46),
            (139,  208),
            (141, 1826),
            (149, 1168),
            (176, 1241),
            (183, 3626),
            (186,  294),
            (192, 2424),
            (202, 4006),
            (212,  670),
            (216, 2550),
            (217,  872),
            (219, 7158),
            (226, 1452),
            (229, 4606),
            (244,  459),


In [55]:
golden_pairs

,,name,ceo
28,1136,1.0,0.5
64,385,1.0,0.5
71,6458,1.0,0.5
113,2695,1.0,0.5
130,708,1.0,0.5
139,2322,1.0,0.5
279,4899,1.0,0.5
313,1369,1.0,0.5
315,2993,1.0,0.5
353,2306,1.0,0.5


In [56]:
logreg = recordlinkage.LogisticRegressionClassifier()
logreg.fit(golden_pairs, golden_match_index)
print('Intercept: ', logreg.intercept)
print('Coefficient: ', logreg.coefficients)

Intercept:  -0.8655910874090981
Coefficient:  [2.33481273e-05 1.16740636e-05]


In [57]:
result_logreg = logreg.predict(pairs)
len(result_logreg)

0

In [58]:
recordlinkage.recall(true_links, result_logreg)

0.0

In [59]:
recordlinkage.precision(true_links, result_logreg)

ZeroDivisionError: division by zero

In [9]:
f_score_reg = recordlinkage.fscore(true_links, result_logreg)
print('Fscore:', f_score_reg)

0.19075144508670522

In [10]:
svm = recordlinkage.SVMClassifier()
svm.fit(golden_pairs, golden_match_index)

In [11]:
result_svm = svm.predict(pairs)
len(result_svm)

939

In [14]:
fscore_svm = recordlinkage.fscore(true_links, result_svm)
print('Fscore:', fscore_svm)

Fscore: 0.19075144508670522


In [23]:
random_dio = [(544, 17), (294, 1842), (504, 4376), (24, 6250), (173, 925), (107, 4021), (235, 4299), (559, 1234), (116, 3818), (191, 1351), (607, 6161), (398, 1672), (575, 1112), (596, 14), (175, 267), (356, 3002), (100, 2673), (414, 29), (133, 46), (538, 593), (55, 5686), (34, 1126), (73, 4120), (604, 454), (495, 4710), (425, 16), (506, 1367), (397, 4213), (515, 2026), (324, 164), (539, 10), (246, 6490), (276, 196), (121, 4885), (366, 453), (149, 1168), (603, 654), (491, 333), (396, 1662), (24, 603), (533, 439), (54, 1134), (463, 665), (254, 2444), (580, 2223), (55, 4109), (208, 285), (602, 654), (253, 3690), (90, 2178), (402, 1899), (550, 4781), (512, 2718), (176, 1241), (587, 3315), (342, 2216), (574, 352), (415, 1073), (326, 1001), (56, 201), (271, 182), (531, 5566), (271, 4754), (442, 1341), (507, 4059), (582, 2999), (323, 487), (540, 340), (465, 5004), (552, 3887), (345, 5992), (394, 1355), (90, 230), (217, 872), (234, 3014), (39, 4), (85, 57), (283, 3982), (605, 4341), (592, 4397), (570, 91), (407, 34), (23, 941), (386, 2954), (28, 6186), (101, 4448), (63, 3190), (160, 6390), (369, 1853), (135, 954), (29, 1742), (280, 4116), (535, 2143), (311, 3533), (612, 696), (529, 3868), (499, 557), (281, 1657), (112, 4305), (548, 3933), (179, 1903), (83, 5930), (431, 4988), (212, 670), (200, 7144), (330, 427), (18, 638), (612, 458), (48, 2834), (226, 1452), (364, 231), (583, 3285), (216, 2550), (547, 63), (122, 2485), (570, 2747), (128, 20), (478, 1897), (154, 520), (365, 231), (229, 4606), (573, 1119), (391, 45), (23, 3488), (419, 2385), (494, 3514), (64, 385), (204, 2127), (484, 1734), (118, 3745), (107, 134), (421, 7139), (198, 5167), (423, 2193), (575, 2945), (254, 4630), (569, 6241), (152, 3503), (437, 303), (551, 1979), (114, 3773), (21, 897), (337, 570), (448, 5355), (141, 1826), (139, 208), (571, 6076), (156, 5505), (317, 2582), (45, 3348), (26, 526), (463, 1), (56, 6782), (381, 23), (44, 1123), (120, 1191), (244, 459), (289, 343), (459, 5136), (27, 6581), (45, 6026), (58, 479), (60, 5782), (365, 2), (32, 148), (519, 1426), (219, 7158), (352, 727), (458, 5007), (393, 6254), (50, 1908), (527, 4011), (591, 3051), (383, 7050), (178, 469), (40, 897), (72, 1344), (152, 5328), (436, 6693), (416, 397), (219, 2684), (414, 3832), (346, 6348), (571, 959), (226, 6027), (312, 4383), (561, 50), (6, 4922), (5, 4749), (613, 4985), (20, 4898), (231, 4337), (443, 761), (576, 947), (227, 3448), (380, 2052), (309, 1180), (368, 7073), (474, 3050), (32, 1115), (438, 290), (105, 1052), (318, 710), (536, 2162), (113, 452), (584, 3744), (252, 575), (534, 2532), (265, 1523), (530, 3630), (380, 5311), (142, 4496), (519, 5112), (117, 1321), (266, 584), (585, 836), (428, 389), (133, 529), (361, 3541), (369, 2009), (258, 619), (581, 4979), (412, 6401), (155, 506), (475, 4166), (610, 5168), (164, 97), (202, 4279), (524, 2930), (428, 450), (470, 4268), (175, 1834), (543, 17), (416, 107), (68, 205), (517, 5741), (224, 300), (553, 70), (195, 3282), (503, 359), (24, 88), (423, 6373), (80, 2862), (265, 6175), (265, 2056), (300, 1148), (182, 2725), (373, 1752), (246, 252), (204, 216), (132, 2836), (370, 344), (138, 1585), (583, 3380), (476, 2066), (399, 3656), (153, 4093), (52, 4192), (239, 1893), (577, 620), (252, 6151), (416, 89), (192, 2424), (507, 3516), (347, 2318), (98, 209), (148, 1318), (185, 2434), (21, 1337), (457, 2052), (291, 2167), (218, 4271), (18, 2803), (213, 4702), (285, 5852), (546, 5716), (615, 5398), (598, 843), (282, 1562), (419, 3906), (328, 2111), (558, 5360), (453, 2438), (62, 6055), (88, 7067), (65, 1139), (171, 6638), (257, 82), (381, 471), (432, 1709), (46, 2832), (375, 2018), (531, 4929), (263, 780), (222, 617), (307, 296), (617, 1716), (381, 725), (221, 2705), (600, 1641), (180, 5517), (160, 1259), (548, 1050), (126, 2039), (111, 578), (463, 899), (186, 294), (162, 6665), (252, 762), (363, 3568), (201, 4839), (429, 4670), (603, 5053), (589, 7062), (400, 2397), (114, 4009), (522, 4642), (51, 6091), (488, 3914), (618, 5147), (374, 1494), (123, 803), (546, 544), (493, 2924), (88, 2608), (555, 4690), (350, 3410), (479, 6924), (291, 12), (233, 3124), (104, 1451), (309, 291), (165, 1197), (541, 340), (78, 3320), (210, 637), (389, 4025), (562, 1935), (82, 7087), (230, 5050), (439, 4144), (11, 2798), (434, 1151), (287, 3604), (271, 1738), (22, 1394), (438, 4144), (385, 6908), (465, 985), (549, 2497), (95, 98), (325, 3923), (548, 4142), (331, 2332), (389, 626), (369, 261), (1, 5560), (52, 85), (131, 1772), (492, 1118), (502, 5303), (96, 1990), (513, 5270), (247, 2837), (498, 2885), (22, 5104), (255, 342), (313, 1369), (382, 1861), (108, 2458), (350, 1822), (218, 5349), (286, 438), (15, 2195), (201, 399), (489, 4766), (217, 5349), (563, 1188), (609, 2181), (194, 5325), (149, 3769), (611, 2125), (516, 272), (494, 3717), (5, 457), (539, 1031), (19, 2565), (86, 4329), (588, 4084), (78, 293), (396, 494), (262, 1034), (420, 4162), (161, 3396), (174, 1834), (561, 3873), (49, 4567), (31, 3322), (17, 5023), (545, 1790), (482, 672), (52, 6748), (598, 634), (419, 4838), (209, 363), (316, 5518), (429, 4354), (122, 7098), (203, 1358), (144, 859), (611, 1736), (43, 6235), (95, 820), (195, 4179), (304, 2157), (119, 92), (572, 4482), (582, 6139), (183, 3626), (453, 1819), (151, 1920), (388, 1200), (460, 722), (518, 4039), (594, 6953), (264, 780), (247, 6509), (202, 4006), (565, 714), (569, 773), (422, 6110), (128, 1009), (495, 542), (126, 1170), (143, 6109), (343, 6066), (329, 103), (33, 6155), (29, 700), (48, 3824), (561, 944), (474, 241), (86, 5441), (564, 918), (7, 3842), (46, 6180), (541, 677), (540, 5852), (508, 1056), (379, 4140), (395, 5707), (473, 175), (134, 393), (260, 7113), (565, 1175), (215, 6647), (440, 3268), (427, 2778), (5, 6728), (576, 527), (278, 751), (69, 5438), (566, 959), (456, 26), (297, 5060), (516, 4288), (97, 6771), (30, 5129), (404, 1209), (29, 1247), (485, 6300), (347, 447), (41, 1518), (390, 5636), (501, 774), (616, 962), (514, 7075), (422, 6272), (442, 3631), (533, 6827), (426, 4859), (157, 4832), (184, 255), (115, 4009), (11, 27), (230, 5081), (295, 6481), (443, 4804), (240, 986), (487, 4272), (348, 6328), (242, 2343), (60, 1718), (85, 251), (345, 881), (433, 3001), (223, 2705), (68, 5277), (274, 2346), (314, 6377), (503, 3395), (281, 355), (486, 7161), (144, 536), (387, 320), (581, 5557), (532, 1593), (609, 1795), (553, 6765), (375, 2258), (205, 3290), (581, 1304), (357, 78), (498, 1192), (569, 566), (89, 6067), (146, 3653), (206, 3290), (169, 4215), (405, 2451), (412, 866), (192, 4074), (402, 1408), (238, 1937), (478, 5380), (277, 178), (553, 1054), (145, 5176), (4, 6549), (439, 6708), (490, 3244), (241, 5479), (169, 4789), (539, 842), (340, 2042), (35, 3284), (192, 1125), (98, 3509), (34, 3811), (388, 2051), (148, 3971), (131, 139), (325, 65), (475, 1580), (275, 196), (552, 2607), (179, 5250), (25, 5964), (434, 1709), (31, 2752), (172, 433), (91, 4756), (92, 2901), (177, 2689), (319, 1587), (518, 306), (444, 6044), (411, 2365), (579, 601), (129, 1008), (209, 2605), (397, 4870), (501, 2772), (425, 2174), (443, 2563), (136, 5435), (328, 3060), (371, 6694), (403, 3102), (19, 2803), (93, 230), (93, 176), (457, 3707), (229, 3130), (307, 1559), (278, 318), (221, 171), (61, 4026), (372, 1752), (60, 562), (353, 2306), (387, 3674), (269, 7082), (244, 5739), (371, 6878), (164, 2974), (399, 5651), (95, 3243), (505, 3015), (384, 49), (476, 1643), (537, 1238), (574, 4913), (80, 5527), (12, 5905), (471, 5286), (67, 992), (248, 3846), (145, 1693), (340, 3965), (339, 1104), (508, 128), (523, 6352), (44, 221), (418, 977), (16, 638), (358, 4752), (183, 2159), (99, 4473), (528, 1682), (138, 548), (53, 5710), (411, 2345), (446, 903), (118, 1194), (167, 731), (163, 2843), (421, 6199), (321, 2333), (511, 2176), (40, 4951), (320, 6423), (58, 2411), (469, 2511), (2, 2791), (190, 6061), (484, 6484), (556, 1054), (599, 2626), (319, 3804), (544, 6740), (326, 3044), (96, 2720), (200, 332), (330, 44), (362, 767), (9, 48), (614, 5148), (557, 2371), (587, 4893), (602, 2270), (243, 3788), (445, 1639), (173, 267), (391, 7151), (467, 1430), (502, 3498), (109, 1924), (500, 4206), (411, 299), (568, 910), (385, 5519), (520, 1024), (160, 506), (194, 6427), (290, 2364), (280, 2937), (403, 4193), (607, 586), (334, 1400), (83, 891), (483, 1291), (314, 2452), (76, 1392), (432, 3936), (462, 104), (514, 6439), (244, 7016), (103, 4473), (59, 2911), (449, 4729), (550, 1443), (323, 5443), (111, 1924), (63, 416), (608, 2003), (464, 96), (174, 6870), (49, 4315), (207, 442), (193, 9), (202, 5990), (409, 3420), (300, 336), (454, 6812), (106, 4021), (301, 4024), (279, 751), (214, 2010), (519, 5082), (535, 1287), (513, 1003), (562, 5388), (413, 3820), (493, 5401), (554, 4690), (9, 27), (404, 4194), (306, 1878), (496, 1769), (318, 6009), (190, 2352), (594, 2256), (248, 2123), (339, 3313), (347, 6641), (124, 73), (147, 2267), (408, 3687), (327, 3596), (302, 6160), (497, 3167), (71, 6458), (503, 1661), (558, 2021), (170, 4308), (84, 251), (329, 4972), (379, 1208), (123, 589), (245, 3066), (510, 2528), (451, 3120), (148, 1857), (37, 3570), (270, 4802), (144, 40), (250, 7072), (256, 407), (232, 7010), (549, 259), (224, 298), (74, 6239), (10, 946), (577, 5557), (173, 519), (221, 202), (547, 3033), (49, 647), (103, 125), (486, 382), (73, 1344), (556, 2092), (277, 5981), (288, 4352), (315, 2993), (306, 4396), (362, 3568), (215, 1166), (473, 786), (38, 160), (481, 5013), (472, 786), (448, 2487), (82, 6299), (238, 4096), (335, 2268), (366, 1923), (211, 824), (146, 1289), (66, 2433), (521, 1037), (81, 1725), (97, 155), (614, 4348), (248, 884), (341, 3994), (187, 1159), (352, 5801), (70, 503), (140, 59), (132, 4335), (477, 6484), (452, 782), (383, 1751), (435, 5205), (525, 317), (162, 1676), (93, 3792), (433, 93), (373, 4078), (199, 6872), (486, 2246), (74, 3713), (466, 1306), (605, 6321), (250, 4942), (308, 1180), (578, 620), (615, 5488), (388, 1323), (229, 6056), (66, 2976), (177, 5097), (400, 6992), (547, 990), (303, 1476), (524, 2895), (579, 766), (441, 6568), (579, 4051), (427, 4205), (99, 3436), (302, 1872), (417, 2131), (317, 1705), (249, 2449), (26, 262), (236, 3043), (10, 3056), (294, 613), (617, 3791), (398, 6946), (9, 2113), (77, 5604), (586, 2965), (390, 7160), (424, 6662), (245, 4469), (239, 4830), (222, 4688), (113, 5242), (293, 3941), (299, 1261), (156, 247), (267, 1475), (327, 5417), (236, 1290), (157, 180), (322, 2902), (176, 295), (593, 174), (487, 382), (354, 4371), (183, 1827), (260, 86), (189, 1814), (33, 30), (410, 99), (75, 3343), (61, 2279), (224, 595), (483, 2978), (69, 503), (163, 2848), (203, 1025), (285, 6058), (251, 600), (455, 2933), (461, 3897), (338, 2247), (59, 125), (207, 830), (351, 643), (147, 505), (333, 2342), (336, 5531), (8, 3056), (471, 157), (208, 1807), (509, 3106), (273, 2697), (449, 43), (551, 631), (358, 1852), (259, 3867), (532, 5342), (109, 578), (186, 6733), (127, 431), (181, 5201), (255, 3230), (532, 79), (181, 693), (284, 5026), (525, 922), (596, 5006), (268, 5554), (179, 1925), (559, 439), (319, 1456), (62, 1964), (125, 5051), (556, 70), (520, 4097), (189, 1045), (571, 2852), (268, 1020), (340, 6520), (17, 5510), (124, 787), (269, 95), (601, 2616), (492, 114), (316, 1919), (597, 4913), (213, 670), (45, 5948), (101, 704), (604, 6587), (227, 5279), (310, 3854), (35, 5369), (355, 1838), (332, 2502), (206, 2770), (613, 1364), (220, 6618), (408, 34), (407, 2547), (431, 1497), (372, 4078), (342, 154), (452, 2196), (102, 146), (245, 1868), (363, 6839), (134, 6356), (448, 87), (383, 1108), (597, 2256), (489, 5116), (237, 4807), (282, 4992), (442, 995), (53, 6584), (491, 4617), (290, 1676), (308, 3437), (155, 1259), (187, 793), (32, 6687), (223, 298), (528, 2425), (415, 305), (266, 5356), (570, 1286), (198, 234), (459, 6209), (334, 7156), (72, 498), (350, 4875), (435, 6974), (13, 5420), (501, 6867), (496, 187), (14, 3451), (43, 2097), (135, 4991), (333, 7156), (458, 5034), (35, 1459), (492, 3187), (232, 5549), (54, 6923), (279, 4899), (523, 5896), (578, 4301), (109, 6340), (343, 4286), (450, 3675), (472, 2580), (471, 2511), (110, 749), (158, 6178), (424, 6442), (504, 6287), (406, 3287), (218, 652), (607, 7027), (170, 4306), (389, 6415), (269, 2873), (480, 2530), (130, 708), (409, 1513), (560, 2251), (374, 4633), (376, 5091), (150, 6629), (124, 1496), (150, 486), (368, 2651), (376, 873), (413, 7076), (104, 1242), (116, 1272), (392, 25), (84, 1533), (288, 6), (522, 1324), (50, 6091), (59, 4473), (129, 100), (589, 3210), (223, 300), (274, 5507), (462, 3977), (470, 4368), (538, 2230), (154, 6390), (616, 743), (510, 788), (36, 3), (77, 4842), (567, 1928), (568, 3461), (289, 6), (444, 6333), (174, 1350), (341, 5025), (16, 5631), (616, 1051), (94, 5313), (231, 2281), (176, 5473), (298, 4137), (255, 2435), (182, 490), (47, 1710), (75, 6239), (151, 6638), (228, 1789), (351, 3255), (480, 1482), (140, 5166), (311, 1467), (212, 824), (72, 5069), (386, 4463), (515, 579), (196, 2409), (576, 2369), (56, 2086), (417, 397), (215, 2319), (20, 2565), (354, 489), (217, 1963), (393, 740), (239, 54), (303, 6418), (482, 1298), (311, 238), (514, 105), (401, 3102), (172, 2080), (299, 2712), (468, 5756), (264, 2056), (591, 1260), (42, 5570), (153, 6888), (112, 3455), (64, 2122), (585, 6258), (103, 2911), (591, 582), (38, 2691), (606, 5049), (111, 5242), (256, 5874), (509, 219), (249, 1539), (180, 5109), (237, 5107), (497, 4659), (490, 825), (508, 6363), (15, 239), (460, 4001), (81, 891), (113, 2695), (16, 2803), (304, 818), (286, 1649), (360, 3354), (257, 2562), (89, 118), (440, 6794), (317, 2630), (526, 1458), (6, 5506), (249, 2837), (267, 1724), (159, 2046), (312, 4237), (165, 559), (270, 5571), (482, 3969), (120, 4148), (338, 583), (593, 6570), (600, 3485), (584, 3285), (590, 5117), (589, 1409), (405, 2700), (517, 4596), (47, 3177), (554, 2310), (7, 6186), (101, 5362), (468, 1430), (313, 473), (171, 1920), (27, 4926), (367, 3662), (42, 1199), (237, 4299), (537, 1244), (427, 1055), (610, 1226), (21, 127), (414, 2868), (296, 6387), (454, 1864), (586, 4549), (464, 4454), (355, 1740), (324, 2704), (536, 1287), (566, 4940), (225, 3824), (384, 4463), (200, 2886), (188, 4956), (392, 4651), (588, 7066), (612, 6582), (609, 1541), (122, 877), (141, 5632), (437, 4696), (286, 2366), (4, 3048), (546, 593), (73, 297), (297, 686), (346, 862), (150, 2380), (133, 2540), (210, 6791), (467, 6234), (199, 3847), (89, 2608), (166, 559), (12, 1433), (142, 75), (281, 6207), (87, 1447), (142, 7119), (355, 5736), (365, 3779), (356, 78), (216, 4679), (79, 384), (557, 3526), (0, 4749), (258, 3867), (77, 345), (315, 2758), (423, 3076), (257, 5874), (446, 2023), (51, 1908), (394, 5935), (30, 6899), (477, 5628), (580, 5070), (349, 5676), (537, 760), (261, 1103), (564, 2812), (391, 2340), (436, 5319), (430, 6316), (461, 3918), (106, 4434), (543, 4730), (409, 2744), (219, 4537), (175, 4641), (182, 5847), (168, 5455), (66, 6136), (608, 1736), (595, 378), (512, 7061), (63, 2099), (76, 3343), (193, 6427), (456, 2093), (226, 4443), (149, 505), (241, 6864), (344, 154), (564, 11), (57, 479), (282, 2421), (8, 1109), (57, 2088), (335, 5215), (542, 4589), (105, 4708), (534, 2162), (152, 4468), (464, 804), (590, 6512), (228, 1249), (121, 4148), (0, 5258), (480, 3292), (382, 1525), (330, 1275), (549, 6200), (25, 1155), (495, 6298), (386, 5680), (14, 3273), (426, 4941), (559, 3780), (3, 4477), (253, 2393), (361, 767), (487, 7161), (313, 5543), (50, 1574), (377, 2405), (13, 4905), (156, 362), (94, 2019), (417, 89), (104, 1284), (240, 495), (26, 136), (190, 3755), (430, 4354), (233, 1030), (158, 4445), (390, 1084), (357, 3831), (378, 4774), (61, 956), (251, 1308), (542, 5465), (588, 6043), (11, 48), (601, 7127), (272, 3723), (378, 6709), (433, 2033), (3, 1005), (428, 1461), (353, 47), (15, 5429), (263, 1888), (284, 6563), (100, 3803), (555, 6773), (12, 56), (130, 861), (242, 1937), (108, 2925), (305, 2157), (277, 2674), (2, 5315), (567, 685), (334, 6513), (298, 7078), (566, 289), (118, 5444), (181, 218), (425, 2020), (451, 2668), (488, 905), (98, 1282), (606, 2204), (545, 6144), (141, 837), (335, 2631), (455, 1609), (364, 415), (543, 779), (187, 2483), (595, 1665), (410, 2664), (395, 4338), (296, 4053), (211, 4402), (359, 180), (490, 1082), (530, 2820), (276, 524), (259, 86), (562, 2212), (572, 5633), (79, 1212), (505, 5724), (167, 6574), (263, 1523), (36, 6216), (318, 3923), (530, 7152), (370, 972), (324, 1297), (441, 5423), (119, 931), (112, 5068), (331, 3885), (515, 215), (496, 1651), (292, 4591), (402, 851), (469, 175), (606, 1453), (572, 2466), (415, 3671), (41, 3623), (28, 1136), (599, 5317), (197, 3878), (42, 321), (211, 670), (7, 1019), (102, 1186), (186, 5108), (506, 5307), (134, 145), (39, 4902), (370, 797), (558, 5210), (280, 110), (483, 5816), (450, 6218), (166, 731), (40, 127), (460, 3857), (447, 3524), (563, 1510), (225, 2834), (359, 2888), (34, 6786), (436, 5713), (251, 935), (214, 3659), (139, 2322), (360, 21), (310, 4237), (197, 2303), (53, 2822), (110, 5590), (474, 3026), (70, 4680), (592, 2286), (1, 2536), (467, 1306), (466, 681), (90, 176), (287, 462), (401, 1899), (117, 841), (404, 3990), (441, 42), (585, 4565), (301, 6943), (10, 4323), (367, 153), (305, 2069), (406, 1468), (236, 5052), (137, 5974), (147, 1168), (526, 7129), (4, 195), (37, 2946), (78, 5940), (518, 3549), (298, 1488), (295, 5667), (243, 3008), (499, 6411), (528, 908), (155, 3046), (511, 3115), (159, 247), (125, 2039), (169, 130), (529, 6989), (205, 6391), (527, 185), (479, 6837), (354, 2201), (392, 1309), (67, 6136), (329, 1996), (13, 322), (198, 624), (196, 5938), (300, 3491), (51, 1574), (513, 706), (338, 6495), (344, 4666), (333, 1400), (76, 6182), (161, 6339), (264, 1523), (538, 544), (418, 268), (127, 2011), (573, 6171), (220, 1030), (27, 1257), (178, 3138), (596, 217), (521, 796), (560, 4056), (510, 4395), (115, 3088), (87, 1348), (500, 3446), (143, 2910), (604, 3326), (168, 660), (387, 885), (447, 1828), (598, 3819), (291, 3248), (462, 2949), (420, 107), (71, 4704), (297, 4045), (207, 354), (256, 5591), (188, 2221), (309, 2014), (168, 558), (332, 3885), (283, 4650), (191, 1063), (68, 4050), (445, 2669), (139, 1058), (273, 2990), (8, 6020), (216, 1963), (468, 6234), (320, 2265), (105, 661), (511, 3037)]